In [1]:
# basic imports
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA

# demos for visualization
from nilearn import datasets
from nilearn import plotting
from nilearn.connectome import GroupSparseCovariance
from nilearn.connectome import ConnectivityMeasure

# for ML
from sklearn.metrics import mutual_info_score

from sklearn import svm

# os library
from os.path import exists, join

C:\ProgramData\Anaconda3\envs\SLHaka\lib\site-packages\nilearn\datasets\__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
data_path = join(".", "data") if exists(join(".", "data")) \
    else join("..", "input", "statistical-learning-sapienza-spring-2021")

In [3]:
train = pd.read_csv(join(data_path, 'train.csv'))
test = pd.read_csv(join(data_path, 'test.csv'))

In [4]:
def get_data(entry):
    id = entry.id
    variables = entry[['var1', 'var2', 'var3']].to_numpy()
    timeseries = entry[5 if "y" in entry.index else 4:].to_numpy(dtype=float).reshape((115, 116), order="F")
    if "y" in entry.index: return id, entry.y, variables, timeseries
    else: return id, variables, timeseries

In [5]:
time_series = train.apply(lambda row: get_data(row)[-1], axis=1).to_list()

In [6]:
time_series_test = test.apply(lambda row: get_data(row)[-1], axis=1).to_list()

In [23]:
import tensorflow as tf
from tensorflow.keras import layers

class BlackEmbedding(tf.keras.Model):

  def __init__(self):
    super(BlackEmbedding, self).__init__()
    self.inputs = layers.Input(shape=(115,116))
    self.conv1 = layers.Conv2D(32, kernel_size=(3, 116), activation=tf.nn.relu)
    self.batch_norm1 = layers.BatchNormalization()
    self.conv2 = layers.Conv2D(16, kernel_size=(3, 116), activation=tf.nn.relu)
    self.batch_norm2 = layers.BatchNormalization()
    self.conv3 = layers.Conv2D(16, kernel_size=(3, 116), activation=tf.nn.relu)
    self.batch_norm3 = layers.BatchNormalization()
    self.dense1 = layers.Dense(64, activation=tf.nn.relu)
    self.dense2 = layers.Dense(64, activation=tf.nn.softmax)

    # self.svm_model = svm.SVR()

  def call(self, inputs, training=False):
    feature_map = self.batch_norm1(self.conv1(inputs))
    feature_map = self.batch_norm2(self.conv2(feature_map))
    feature_map = self.batch_norm3(self.conv3(feature_map))
    feature_map = self.dense1(feature_map)
    prediction = self.dense2(feature_map)

    """
    if training:
        self.svm_model.fit(embedding, y_train)

    # make prediction
    prediction = self.svm_model.predict(embedding)
    """

    return prediction

model = BlackEmbedding()

In [24]:
model.compile(optimizer="Adam", loss="mse")

In [25]:
model.fit(x=np.array(time_series), y=np.array(train.y), batch_size=32)

ValueError: in user code:

    C:\Users\dario\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:855 train_function  *
        return step_function(self, iterator)
    <ipython-input-23-a5eee9475a38>:21 call  *
        feature_map = self.batch_norm1(self.conv1(inputs))
    C:\Users\dario\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\base_layer.py:1013 __call__  **
        input_spec.assert_input_compatibility(self.input_spec, inputs, self.name)
    C:\Users\dario\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\input_spec.py:230 assert_input_compatibility
        raise ValueError('Input ' + str(input_index) + ' of layer ' +

    ValueError: Input 0 of layer conv2d_8 is incompatible with the layer: : expected min_ndim=4, found ndim=3. Full shape received: (None, 115, 116)


In [15]:
np.array(time_series).shape

(600, 115, 116)

In [17]:
list(map(np.array, time_series))

[array([[-0.140642  ,  0.075528  ,  0.056853  , ..., -0.29544601,
         -0.97051001, -0.137205  ],
        [-0.38816199,  0.24003799,  0.011811  , ..., -0.11126   ,
         -0.99557799, -0.18176   ],
        [-0.57418102,  0.334544  ,  0.03659   , ...,  0.128346  ,
         -0.67245603, -0.083906  ],
        ...,
        [-0.042294  , -0.068071  ,  0.653584  , ..., -0.58853298,
         -0.94182199, -0.98057097],
        [ 0.057394  , -0.011607  ,  0.66784602, ..., -0.499975  ,
         -1.303581  , -0.37752599],
        [ 0.116523  ,  0.17505801,  0.241915  , ..., -0.28508601,
         -1.13002598,  0.16824301]]),
 array([[ 0.023754  , -0.119165  , -0.15271699, ..., -0.51899803,
          1.36734903, -0.90352201],
        [-0.105455  , -0.157781  , -0.200076  , ..., -0.76688701,
          1.79039395, -0.141947  ],
        [-0.24502499, -0.153842  , -0.141443  , ..., -0.60004503,
          1.53425395,  0.92580402],
        ...,
        [-0.057136  ,  0.007818  ,  0.29323599, ...,  

In [36]:
pd.concat([test.iloc[:, :4], pd.DataFrame(correlation_test)], axis=1)

,id,var1,var2,var3,0,1,2,3,4,5,...,6660,6661,6662,6663,6664,6665,6666,6667,6668,6669
0,17,32,A,A,0.023671,-0.007289,-0.009024,0.007298,-0.003480,0.000240,...,0.063989,-0.048858,0.054061,-0.228254,-0.062239,0.001865,0.103494,-0.044311,0.101675,-0.007015
1,19,11,C,A,0.026247,0.023898,-0.039581,-0.010329,-0.016104,-0.005682,...,0.086398,0.191093,0.095374,-0.209788,0.026410,-0.056142,0.075483,0.030596,-0.038504,-0.118798
2,25,17,A,C,0.011462,-0.002216,0.007523,-0.003675,0.020650,0.006799,...,0.066654,0.005996,0.099345,0.158647,0.004112,0.244222,-0.036526,-0.177120,0.026056,-0.026664
3,32,17,A,C,-0.000732,-0.006386,-0.020390,-0.014332,0.017277,0.001105,...,0.087192,-0.120249,-0.001533,0.030498,0.177132,-0.031438,0.083403,0.104198,0.042981,0.032002
4,33,16,A,C,0.055043,-0.020623,-0.046023,0.012963,-0.024777,0.064455,...,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,777,20,A,C,-0.003031,0.007281,-0.024983,-0.017637,0.044618,-0.008667,...,0.140488,-0.114554,-0.043859,0.306518,-0.035661,0.032000,-0.014286,-0.049970,0.145010,0.239272
195,783,24,A,C,0.012482,0.003675,-0.025090,0.010962,0.035677,0.012848,...,0.246349,0.081832,0.060359,-0.243860,-0.027795,0.073448,-0.095433,0.182376,-0.182050,0.381747
196,784,24,A,C,0.059117,-0.019442,0.004688,-0.003915,0.041571,-0.025616,...,0.288217,0.174798,0.001413,0.005653,-0.174478,0.037918,-0.085111,0.042151,-0.294607,0.225792
197,792,29,C,C,0.019908,-0.000896,-0.017489,-0.028130,0.023437,0.000375,...,0.248101,-0.018863,0.092836,0.503031,-0.266960,0.069081,0.307400,-0.216884,0.281473,0.112249


In [40]:
y_train = train.loc[:, 'y']
X_train = train.drop(columns=['y'])

In [11]:
pd.concat([train.iloc[:, :4], pd.DataFrame(time_series)], axis=1).values

ValueError: Must pass 2-d input. shape=(600, 115, 116)

In [55]:
from sklearn.decomposition import KernelPCA
transformer = KernelPCA(kernel='rbf')
X_transformed = transformer.fit_transform(correlation_matrices)
X_transformed.shape

(600, 599)

In [56]:
X_trans_test = transformer.transform(correlation_test)

In [57]:
pd.concat([X_train.iloc[:, :4], y_train, pd.DataFrame(X_transformed)], axis=1).to_csv('train_pca.csv', index=False)
pd.concat([test.iloc[:, :4], pd.DataFrame(X_trans_test)], axis=1).to_csv('test_pca.csv', index=False)
